In [1]:
import pandas as pd

In [5]:
# leer archivo CSV
df = pd.read_csv('ventas.csv')

In [6]:
# verificar las primeras filas del DataFrame
print(df.head())

   nvoice_id entity   country  issue_date currency  net_amount  tax_amount
0          1   E001   ECUADOR  2024-05-03      USD        1000         120
1          2   E002  COLOMBIA  2024-05-04      COP     2500000      475000
2          3   E003      PERU  2024-05-05      PEN        3500         630
3          4   E001   ECUADOR  2024-05-06      USD         800          96


In [ ]:
#(1) CLEAN: limpieza y estandarización 
print("Limpieza de datos...")
df.columns = df.columns.str.strip().str.lower()  # estandarizar nombres
df['country'] = df['country'].str.strip().str.upper()  # mayúsculas
df['currency'] = df['currency'].str.strip().str.upper()

df.head()

Limpieza de datos...


,nvoice_id,entity,country,issue_date,currency,net_amount,tax_amount
0,1,E001,ECUADOR,2024-05-03,USD,1000,120
1,2,E002,COLOMBIA,2024-05-04,COP,2500000,475000
2,3,E003,PERU,2024-05-05,PEN,3500,630
3,4,E001,ECUADOR,2024-05-06,USD,800,96


In [ ]:
# (2) TRANSFORM: conversión a USD 
fx_to_usd = {"USD":1.0, "COP":0.00026, "PEN":0.27}

print("Convirtiendo montos a USD...")
df['net_usd'] = df.apply(lambda x: x['net_amount'] * fx_to_usd.get(x['currency'], 1), axis=1)
df['tax_usd'] = df.apply(lambda x: x['tax_amount'] * fx_to_usd.get(x['currency'], 1), axis=1)
df['total_usd'] = df['net_usd'] + df['tax_usd']

df.head()

Convirtiendo montos a USD...


,nvoice_id,entity,country,issue_date,currency,net_amount,tax_amount,net_usd,tax_usd,total_usd
0,1,E001,ECUADOR,2024-05-03,USD,1000,120,1000.0,120.0,1120.0
1,2,E002,COLOMBIA,2024-05-04,COP,2500000,475000,650.0,123.5,773.5
2,3,E003,PERU,2024-05-05,PEN,3500,630,945.0,170.1,1115.1
3,4,E001,ECUADOR,2024-05-06,USD,800,96,800.0,96.0,896.0


In [ ]:
# (3) LOAD: Agregar por país 
print("Agregando totales por país...")
resumen = df.groupby('country')[['net_usd', 'tax_usd', 'total_usd']].sum().reset_index()
print(resumen)


Agregando totales por país...
    country  net_usd  tax_usd  total_usd
0  COLOMBIA    650.0    123.5      773.5
1   ECUADOR   1800.0    216.0     2016.0
2      PERU    945.0    170.1     1115.1


In [ ]:
# (4) Exportar resultado a CSV
resumen.to_csv("resumen_pais.csv", index=False)
print("Archivo 'resumen_pais.csv' generado exitosamente.")

Archivo 'resumen_pais.csv' generado exitosamente.


In [ ]:
# (5) LOG del proceso
print("\n============== LOG DEL PROCESO ============")
print(f"Registros procesados: {len(df)}")
print("Países incluidos:", ", ".join(resumen['country']))
print("Totales por país:")
print("==========================================")
print(resumen)
print("==========================================")


============== LOG DEL PROCESO ============
Registros procesados: 4
Países incluidos: COLOMBIA, ECUADOR, PERU
Totales por país:
    country  net_usd  tax_usd  total_usd
0  COLOMBIA    650.0    123.5      773.5
1   ECUADOR   1800.0    216.0     2016.0
2      PERU    945.0    170.1     1115.1
